In [1]:
from pynq.overlays.base import BaseOverlay

base = BaseOverlay('base.bit')

base.init_rf_clks()

In [48]:
def set_transmitter_channel(channel, enable, gain, frequency, phaseOff):
    channel.control.enable = enable
    channel.control.gain = gain
    channel.dac_block.MixerSettings['Freq'] = frequency
    channel.dac_block.MixerSettings['PhaseOffset'] = phaseOff

set_transmitter_channel(base.radio.transmitter.channel[0], True, 1, 433, 90)
set_transmitter_channel(base.radio.transmitter.channel[1], True, 1, 433, 0)

In [49]:
number_samples = 1229
sample_frequency = 2458e6
cdata = []
for i in range(0, len(base.radio.receiver.channel)):
    cdata.append(base.radio.receiver.channel[i].transfer(number_samples))

In [50]:
import plotly.graph_objs as go
import numpy as np
import ipywidgets as ipw

def plot_complex_time(data, n=number_samples, fs=sample_frequency, 
                      title='Complex Time Plot'):
    plt_re_temp = (go.Scatter(x = np.arange(0, n/fs, 1/fs),
                              y = np.real(data), name='Real'))
    plt_im_temp = (go.Scatter(x = np.arange(0, n/fs, 1/fs),
                              y = np.imag(data), name='Imag'))
    return go.FigureWidget(data = [plt_re_temp, plt_im_temp],
                           layout = {'title': title, 
                                     'xaxis': {
                                         'title': 'Seconds (s)',
                                         'autorange' : True},
                                     'yaxis': {
                                         'title': 'Amplitude (V)'}})

tfigs = []
for i in range(2, len(base.radio.receiver.channel)):
    tfigs.append(plot_complex_time(
            data=cdata[i], 
            title=''.join(['Time Domain Plot of ADC Channel ', str(i)])))

ipw.VBox(tfigs)

    'data': [{'name': 'Real',
              'type': 'scatter',
              'ui…

In [53]:
window = np.array(np.blackman(number_samples)[:])
wdata = []
wfigs = []

for i in range(2, len(base.radio.receiver.channel)):
    wdata.append(cdata[i]*window)
    wfigs.append(plot_complex_time(
            data=wdata[i-2], 
            title=''.join(['Windowed Plot of ADC Channel ', str(i)])))

ipw.VBox(wfigs)

    'data': [{'name': 'Real',
              'type': 'scatter',
              'ui…

In [ ]:
fdata = []
for i in range(0, len(wdata)):
    fdata.append(np.fft.fftshift(np.fft.fft(wdata[i])))

In [47]:
def plot_complex_spectrum(data, N=number_samples, fs=sample_frequency, 
                          title='Complex Spectrum Plot', units='dBW', fc=0):
    plt_temp = (go.Scatter(x = np.arange(-fs/2, fs/2, fs/N) + fc,
                           y = data, name='Spectrum'))
    return go.FigureWidget(data = plt_temp,
                           layout = {'title': title, 
                                     'xaxis': {
                                         'title': 'Frequency (Hz)',
                                         'autorange': True},
                                     'yaxis': {
                                         'title': units}})

magfigs = []
for i in range(0, len(fdata)):
    magfigs.append(plot_complex_spectrum(
            data=abs(fdata[i]),
            title=''.join(['Frequency Magnitude Plot of ADC Channel ', str(i+2)]),
            units='|Y(f)|',
            fc=round(abs(base.radio.receiver.channel[i].adc_block.MixerSettings['Freq']))*1e6))

ipw.VBox(magfigs)

    'data': [{'name': 'Spectrum',
              'type': 'scatter',
             …

In [40]:
def freq_to_psd(data, N=number_samples, fs=sample_frequency, 
                window=np.array(np.ones(number_samples)[:])):
    psd = (abs(data)**2)/(fs*np.sum(window**2))
    return 10*np.where(psd > 0, np.log10(psd), 0)

psd = []
for i in range(0, len(fdata)):
    psd.append(freq_to_psd(data=fdata[i], window=window))

In [41]:
psdfigs = []
for i in range(0, len(psd)):
    psdfigs.append(plot_complex_spectrum(
            data=psd[i],
            title=''.join(['Power Spectral Density Plot of ADC Channel ',
                           str(i+2)]),
            units='Amplitude (dB)',
            fc=round(abs(base.radio.receiver.channel[
                        i].adc_block.MixerSettings['Freq']))*1e6))

ipw.VBox(psdfigs)

    'data': [{'name': 'Spectrum',
              'type': 'scatter',
             …